## Version 2
### accuracy: 0.9750367
- only delete punctuation + jieba + remove stopwords
- Tokenizer
- TfidfVectorizer
- Ensemble model

### 改进方向
- 存在非中文content，需要处理
- lightgbm参数调优，加入catboost分类器
- 融合 bert-like transformer(prob < x)

In [1]:
import pandas as pd
import string
import re
import jieba
import joblib
from tqdm import tqdm
tqdm.pandas()
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from transformers import PreTrainedTokenizerFast
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

In [2]:
train_dir = './train.csv'
test_dir = './test.csv'
df1 = pd.read_csv(train_dir)
df2 = pd.read_csv(test_dir)

In [3]:
train = df1[['id', 'content', 'topic']]
test = df2[['id', 'content', 'topic']]
test

,id,content,topic
0,181,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,1
1,1763,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1
2,1015,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1
3,7104,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1
4,2427,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1
...,...,...,...
3400,1044,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23
3401,9043,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23
3402,5570,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23
3403,5523,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23


In [4]:
import re
import string
import jieba

def get_stopword():
    with open('./baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f)
    stopwords.update(["年","月","日"])
    return stopwords

def wordopt_cn(text):
    # replace punctuation with space
    no_punct = re.sub(pattern, ' ', text)
    return str(no_punct)

def remove_stopwords(words):
    return (word for word in words if word not in stopwords)

def process_content(content):
    content = wordopt_cn(content)
    words = jieba.cut(content)
    words = remove_stopwords(words)
    content = ' '.join(words)
    content = re.sub(spaces_pattern, ' ', content)
    return content

# Precompile the regular expressions
punctuation = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." + string.punctuation
pattern = re.compile('[%s]' % re.escape(punctuation))
spaces_pattern = re.compile(r'\s+')
stopwords = get_stopword()

In [5]:
train.loc[:,'content'] = train['content'].progress_apply(process_content)
train.to_csv('train_jieba.csv', index=False)

  0%|          | 0/6810 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shens\AppData\Local\Temp\jieba.cache
Loading model cost 0.363 seconds.
Prefix dict has been built successfully.
100%|██████████| 6810/6810 [00:23<00:00, 288.63it/s]


In [6]:
test.loc[:,'content'] = test['content'].progress_apply(process_content)
test.to_csv('test_jieba.csv', index=False)

100%|██████████| 3405/3405 [00:11<00:00, 293.76it/s]


In [7]:
test

,id,content,topic
0,181,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1
1,1763,当地 时间 11 16 习近平 主席 亚太经合组织 APEC 工商 领导人 峰会 发表 书面...,1
2,1015,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1
3,7104,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 2...,1
4,2427,中国 国家 主席 习近平 11 14 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会...,1
...,...,...,...
3400,1044,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23
3401,9043,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23
3402,5570,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23
3403,5523,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23


In [8]:
LOWERCASE = False
VOCAB_SIZE = 100_000

In [9]:
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['content'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

100%|██████████| 6810/6810 [00:03<00:00, 1724.72it/s]


In [10]:
def dummy(text):
    return text
vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

vectorizer.fit(tokenized_texts_test)

vocab = vectorizer.vocabulary_

vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, vocabulary=vocab,
                             analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

X_train = vectorizer.fit_transform(tokenized_texts_train)
joblib.dump(vectorizer, 'vectorizer.pkl')
print(len(vocab))

89684


In [11]:
clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber")
p6 = {
    'n_iter': 1500, 
    'verbose': -1, 
    'objective': 'multiclass', 
    'metric': 'multi_logloss', 
    'learning_rate': 0.05073909898961407,
    'colsample_bytree': 0.726023996436955, 
    'colsample_bynode': 0.5803681307354022, 
    'lambda_l1': 8.562963348932286,
    'lambda_l2': 4.893256185259296, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898,
    'num_class': 24
}
lgb = LGBMClassifier(**p6)

# Creating the ensemble model
ensemble = VotingClassifier(estimators=[
    ('mnb', clf),
    ('sgd', sgd_model),
    ('lgb', lgb)],
    weights=[0.1, 0.45, 0.45],
    voting='soft',
    n_jobs=-1)

In [12]:
Y_train = train['topic'].values
ensemble.fit(X_train, Y_train)
joblib.dump(ensemble, 'ensemble.pkl')

['ensemble.pkl']

In [16]:
tf_test = vectorizer.transform(tokenized_texts_test)
pred = ensemble.predict(tf_test)
prob = ensemble.predict_proba(tf_test)
test["pred"] = pred
test["prob"] = prob.max(axis=1)

In [17]:
different = test[test['topic'] != test['pred']]
different

,id,content,topic,pred,prob
0,181,2023 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人...,1,22,0.699783
7,2861,APEC 峰会 前 公使 提出 四点 核心 利益 中 美 会晤 看 美 表现 一带 一路 国...,1,4,0.590462
40,4395,2023 亚太经合 会 APEC 领袖 峰会 今 15 日于 美国 旧金山 登场 泰国 总理...,1,0,0.473018
43,3236,焦点访谈 把舵 中美关系 领航 亚太 合作 央视网 2023 11 19 22 01 加载 ...,1,4,0.837395
60,3670,亚太 国家 地区 领导人 13 抵达 美国 亚太经合组织 APEC 第三十次 领导人 非正式...,1,4,0.762252
...,...,...,...,...,...
3090,3159,中国 空间站 建造 期 一棒 神舟 十五号 三大 看点 2022 11 30 10 54 5...,21,19,0.947157
3096,7668,2022 11 28 上午 9 时 神舟 十五号 载人 飞行 新闻 发布会 酒泉卫星发射中心...,21,19,0.925985
3099,598,2023 12 31 日晚 国家 主席 习近平 发表 二 〇 二 四年 新年贺词 2013 ...,21,19,0.490701
3127,5424,神舟 十七号 发射成功 续写 飞天 梦想 建设 航天 强国 北京 时间 10 26 11 时...,21,20,0.937831


In [18]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9753303964757709
Precision: 0.9755118853051953
Recall: 0.9753303964757709
F1: 0.9753198082233252
